In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Leitura do arquivo CSV e configuração de exibição de colunas
cota_parlamentar_df = pd.read_csv("cota-parlamentar.csv")
pd.set_option('display.max_columns', None)

In [ ]:

# Visualização inicial dos dados
cota_parlamentar_df.head()
cota_parlamentar_df.info()
cota_parlamentar_df.describe(include="all")

In [ ]:
# Função para converter valores para milhões e substituir erros por zero
def converter_para_milhoes(df, colunas):
    for coluna in colunas:
        try:
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce') / 1e6  # Converte para número e divide por 1e6
            df[coluna].fillna(0, inplace=True)  # Substitui valores nulos por zero
        except Exception as e:
            print(f"Erro ao processar a coluna {coluna}: {e}")
            df[coluna] = 0  # Substitui a coluna inteira por zero em caso de erro
    return df

# Aplicar a função às colunas desejadas
colunas_para_converter = ['vlrdocumento', 'vlrglosa', 'vlrliquido', 'vlrrestituicao']
cota_parlamentar_df = converter_para_milhoes(cota_parlamentar_df, colunas_para_converter)

# Verificar o resultado
print(cota_parlamentar_df.head())

In [ ]:
# Análise 1: Gráfico de barras para visualizar os gastos por partido
cota_partido_df = cota_parlamentar_df.groupby('sgpartido')['vlrliquido'].sum().sort_values(ascending=False).reset_index()

plt.figure(figsize=(12, 6))
bars = plt.bar(cota_partido_df['sgpartido'], cota_partido_df['vlrliquido'], color='skyblue')
plt.xlabel('Partido')
plt.ylabel('Total de Gastos (Milhões R$)')
plt.title('Total de Gastos por Partido')
plt.xticks(rotation=90)

# Adicionar rótulos de dados em cada barra
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, f'{yval:.2f}', va='bottom', ha='center', rotation=90)

plt.show()

In [ ]:
# Análise 2: Gráfico de barras com gastos por parlamentar organizados por partido

# Preparar os dados
top_50_parlamentares = cota_parlamentar_df.groupby(['txnomeparlamentar', 'sgpartido'])['vlrliquido'].sum().reset_index()
top_50_parlamentares = top_50_parlamentares.nlargest(50, 'vlrliquido')
top_50_parlamentares['nome_com_partido'] = top_50_parlamentares['txnomeparlamentar'] + ' - ' + top_50_parlamentares['sgpartido']

# Ordenar os dados para agrupar parlamentares por partido
top_50_parlamentares = top_50_parlamentares.sort_values(by=['sgpartido', 'vlrliquido'], ascending=[True, False])

# Calcular a média dos gastos dos Top 50
media_gastos = top_50_parlamentares['vlrliquido'].median()

# Criar o gráfico
plt.figure(figsize=(16, 8))

# Gráfico de barras
barras = plt.bar(
    top_50_parlamentares['nome_com_partido'], 
    top_50_parlamentares['vlrliquido'], 
    color='lightblue'
)

# Adicionar rótulos de dados
for barra in barras:
    altura = barra.get_height()
    plt.text(
        barra.get_x() + barra.get_width() / 2, 
        altura*0.85, 
        f'{altura:,.2f}', 
        ha='center', 
        va='bottom', 
        fontsize=9, 
        rotation=90
    )

# Adicionar linha da média
plt.axhline(y=media_gastos, color='red', linestyle='-', linewidth=1.5, label=f'Média: {media_gastos:,.2f}')

# Ajustar o gráfico
plt.xlabel('Parlamentares (Agrupados por Partido)', fontsize=12)
plt.ylabel('Total de Gastos (Milhões R$)', fontsize=12)
plt.title('Gastos por Parlamentar (Top 50) organizados por Partido', fontsize=14)
plt.xticks(rotation=90, fontsize=9)
plt.legend()
plt.tight_layout()

plt.show()


In [ ]:
# Análise 3: Gráfico de barras empilhadas para visualização detalhada por estado
cota_partido_estado_df = cota_parlamentar_df.groupby(['sguf', 'sgpartido'])['vlrliquido'].sum().sort_values(ascending=False).reset_index()
top_10_partidos = cota_partido_estado_df.groupby('sgpartido')['vlrliquido'].sum().nlargest(10).index
cota_partido_estado_df_top_10 = cota_partido_estado_df[cota_partido_estado_df['sgpartido'].isin(top_10_partidos)]
heatmap_data = cota_partido_estado_df_top_10.pivot(index='sguf', columns='sgpartido', values='vlrliquido')

data_matrix = heatmap_data.values
states = heatmap_data.index
parties = heatmap_data.columns

plt.figure(figsize=(14, 10))
cax = plt.imshow(data_matrix, cmap='Reds', aspect='auto')
plt.title('Gastos por Partido em Cada Estado (Top 10 Partidos)')
plt.xlabel('Partido')
plt.ylabel('Estado')
plt.xticks(ticks=np.arange(len(parties)), labels=parties, rotation=90)
plt.yticks(ticks=np.arange(len(states)), labels=states)
plt.colorbar(cax, label='Gastos (milhões R$)')

# Adicionar os valores das células
for i in range(len(states)):
    for j in range(len(parties)):
        if not np.isnan(data_matrix[i, j]):
            plt.text(j, i, f'{data_matrix[i, j]:,.0f}', ha='center', va='center', color='black')

plt.show()

In [ ]:
# Análise 4: Gráfico de pizza para distribuição dos tipos de gastos dos partidos que mais gastaram
gastos_partido_tipo_df = cota_parlamentar_df.groupby(['sgpartido', 'txtdescricao'])['vlrliquido'].sum().reset_index()
top_partidos = gastos_partido_tipo_df.groupby('sgpartido')['vlrliquido'].sum().nlargest(3).index

for partido in top_partidos:
    gastos_partido_df = gastos_partido_tipo_df[gastos_partido_tipo_df['sgpartido'] == partido]
    total_gastos = gastos_partido_df['vlrliquido'].sum()
    gastos_partido_df['percentual'] = (gastos_partido_df['vlrliquido'] / total_gastos) * 100
    gastos_outros_df = gastos_partido_df[gastos_partido_df['percentual'] < 5]
    gastos_maiores_df = gastos_partido_df[gastos_partido_df['percentual'] >= 5]
    gastos_outros_total = gastos_outros_df['vlrliquido'].sum()
    gastos_outros = pd.DataFrame([{'sgpartido': partido, 'txtdescricao': 'Outros', 'vlrliquido': gastos_outros_total}])
    gastos_partido_final_df = pd.concat([gastos_maiores_df, gastos_outros], ignore_index=True)

    plt.figure(figsize=(10, 8))
    plt.pie(gastos_partido_final_df['vlrliquido'], labels=gastos_partido_final_df['txtdescricao'], autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
    plt.title(f'Distribuição dos Tipos de Gastos do Partido "{partido}" (Menores de 5% agrupados em "Outros")')
    plt.show()